In [ ]:
## Mount thu muc google drive

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
import os
os.chdir("drive") # Truy cap thu muc drive 
!ls

'20173016-Nguyễn Huy Định-BT1.docx'   hoctap20201	     result.csv
'Colab Notebooks'		      hoctap20202	     rs.csv
 cuatoi				      khác		    'số đt.desktop'
 document			      MISA		     tin-sinh-hoc
 hoctap20191			      Nguyen_Huy_Dinh.pdf
 hoctap20192			      nhap-mon-hoc-may.zip


In [4]:
os.chdir("tin-sinh-hoc")
!ls

dataset				       ML.ipynb  Pfeature.zip
__MACOSX			       model	 src
Miniconda3-py37_4.8.2-Linux-x86_64.sh  Pfeature  tailieu.txt


In [ ]:
## cai dat thu vien Pfeature

In [5]:
os.chdir("Pfeature")
!ls

build		      Pfeature			PKG-INFO  setup.py
Functions_Tables.pdf  Pfeature_Descriptors.pdf	README


In [6]:
! python setup.py install

/usr/lib/python3.7/distutils/dist.py:274: UserWarning: Unknown distribution option: 'zip_safe'
  warnings.warn(msg)
running install
running build
running build_py
running install_lib
creating /usr/local/lib/python3.7/dist-packages/Pfeature
copying build/lib/Pfeature/bin_di.csv -> /usr/local/lib/python3.7/dist-packages/Pfeature
copying build/lib/Pfeature/aaindices.csv -> /usr/local/lib/python3.7/dist-packages/Pfeature
copying build/lib/Pfeature/z_aaindex.csv -> /usr/local/lib/python3.7/dist-packages/Pfeature
copying build/lib/Pfeature/atom.csv -> /usr/local/lib/python3.7/dist-packages/Pfeature
copying build/lib/Pfeature/AAIndexNames.csv -> /usr/local/lib/python3.7/dist-packages/Pfeature
copying build/lib/Pfeature/aaind.txt -> /usr/local/lib/python3.7/dist-packages/Pfeature
copying build/lib/Pfeature/data -> /usr/local/lib/python3.7/dist-packages/Pfeature
copying build/lib/Pfeature/Grantham.csv -> /usr/local/lib/python3.7/dist-packages/Pfeature
copying build/lib/Pfeature/aa_attr_group.cs

In [8]:
## cai dat biopython
!pip install bio

In [9]:
## import thu vien
import os
from Pfeature.pfeature import paac_wp
import pandas as pd
import pickle
from Bio import SeqIO
import csv
import tempfile

In [10]:
def paac(input):
  a = input.rstrip('fasta')
  output = a + 'paac.csv'
  df_out = paac_wp(input, output,4,0.2)
  df_in = pd.read_csv(output)
  return df_in

In [11]:
def loadModel():
  with open('../model/model.pkl', 'rb') as file:
    pickle_model = pickle.load(file)
  return pickle_model 

In [51]:
def fasta_to_Array(input):
  arr = []
  for record in SeqIO.parse(input, "fasta"):
    item = [record.id,str(record.seq)]
    arr.append(item)
  return arr

In [17]:
#input_fname: file fasta
def antiMicroable(input_fname):
  df = paac(input_fname)
  model = loadModel()
  yPred = model.predict(df)
  yPred = pd.DataFrame(yPred,columns=['class'])
  mydict = fasta_to_Array(input_fname)
  df1 = pd.DataFrame(mydict,columns=['ID', 'Seq'])
  result = pd.concat([df1, yPred], axis=1, ignore_index=True)
  result.columns = ["ID", "Seq", "Class"]
  return result

In [49]:
def antiMicroableBySeq(seq):
  filename = 'tmp.fasta'
  with open(filename, "w") as file:
    # Writing data to a file
    file.write(">001\n")
    file.write(seq)
  rs = antiMicroable(filename)
  folder=os.listdir(".")
  for item in folder:
    if item.endswith((".txt",".fasta",".csv")):
        os.remove(item)
  return rs

In [14]:
os.chdir("..")
os.chdir("src")
!ls

antiMicroable.ipynb  example.aac.csv  example.fasta  example.paac.csv  rs.csv


In [15]:
rs = fasta_to_Array('example.fasta')
rs

[['amp5_30_1', Seq('ACSAG')],
 ['amp5_30_2', Seq('AMVGT')],
 ['amp5_30_3', Seq('AMVSS')],
 ['amp5_30_4', Seq('CPFVC')],
 ['amp5_30_5', Seq('SPGGA')],
 ['amp5_30_6', Seq('DDDDDD')],
 ['amp5_30_7', Seq('FRWWHR')],
 ['amp5_30_8', Seq('RKKWFW')],
 ['amp5_30_9', Seq('RRWWRF')],
 ['amp5_30_10', Seq('TVVTNA')]]

In [15]:
!ls

antiMicroable.ipynb  example.aac.csv  example.fasta  rs.csv


In [53]:
antiMicroable('example.fasta')

,ID,Seq,Class
0,amp5_30_1,ACSAG,1
1,amp5_30_2,AMVGT,0
2,amp5_30_3,AMVSS,0
3,amp5_30_4,CPFVC,1
4,amp5_30_5,SPGGA,1
5,amp5_30_6,DDDDDD,0
6,amp5_30_7,FRWWHR,1
7,amp5_30_8,RKKWFW,1
8,amp5_30_9,RRWWRF,1
9,amp5_30_10,TVVTNA,1


In [54]:
antiMicroableBySeq("FRWWHR")

,ID,Seq,Class
0,001,FRWWHR,1
